# Beleg 2: Bilddateien, Bilddarstellung und Bilddiskretisierung

**Name:**
<br>
**Matrikelnr.:**

## Bearbeitungszeitraum
**Bearbeitungsbeginn:** Mi, 17.11.2022
<br>
**Abgabe (in der Übung zu zeigen bis spätestens):** Mi, 05.01.2023


**Hinweise zum Beleg:** 
- Die erfolgreiche Bearbeitung des Belegs ist zwingende Voraussetzung zur Teilnahme an der Klausur. 
- Sofern nicht anders angegeben können die Operationen mit einer Library Ihrer Wahl durchgeführt werden.


### Segmentierung von Mitosen im PHH3

In dieser Aufgabe werden Sie sich mit der Segmentierung von Mitosen auseinandersetzen. Mitosen sind Zellen, die sich gerade in Teilung befinden. Zwar ist dies ist ein natürlicher Prozess, jedoch findet er so selten statt, dass es statistisch unwahrscheinlich ist, eine Mitose in einem histologischen Schnitt zu finden. Innerhalb von Tumorgewebe jedoch ist die Proliferationsrate (sprich die Rate, mit der sich das Gewebe durch Zellteilung vermehrt) deutlich höher. Für diverse Tumorentitäten spielt die Erfassung dieser Proliferationsrate, etwa durch Auszählung von Mitosen, eine wesentliche Rolle in der Bewertung der Malignität.

### Viel Erfolg!

**Aufgabe 1.1**: Schauen Sie sich zunächst die Bilder im Ordner `mitosis_set` an. Hierbei handelt es sich um invasives Mammakarzinom, welches mit der IHC-Färbung für das Protein Phosphohistone-H3 (PHH3) gefärbt wurde. Der braune Ton der Mitosen entsteht durch Diaminobenzidine (DAB) welches an die entsprechenden Antikörper für das PHH3 gebunden ist. Die blaue Färbung der nicht-mitotischen Zellkerne entsteht durch Haematoxylin. 

Die Dateinamen der Bilder enthalten jeweils zwei Informationen: Zum einen die ID des Bilders (eine fortlaufende Nummer beginnend mit 01) sowie die Anzahl der jeweils im Bild enthaltenen Mitosen. Die Angaben sind durch einen Underscore (\_) getrennt. Lesen Sie nun alle Bilder ein. Überlegen Sie sich eine passende Datenstruktur zur Speicherung der jeweiligen Anzahl an Mitosen.

In [ ]:
# 1.1

**Aufgabe 1.2**: Entwickeln Sie ein Verfahren zur Segmentierung der Mitosen. Als Teilschritte sollen sowohl die Segmentierungsmasken als auch die im Originalbild hevorgehobenen Mitosen (entweder, indem das Bild anhand der Maske zugeschnitten oder indem die Mitosen mit einem Polygon markiert wurden) ausgegeben werden. Am Ende soll für jedes Bild die Anzahl an gefundenen Mitosen ausgegeben werden. Beginnen Sie erst mit Aufgabe 1.3, wenn ihr Algorithmus für jedes Bild die korrekte Anzahl an Mitosen findet. 

In [ ]:
# 1.2 

**Aufgabe 1.3**: Lesen Sie nun die Dateien aus dem Ordner `mitosis_test_set` ein, indem Sie erneut ihre zuvor definierte Datenstruktur verwenden. Wenden Sie ihr Verfahren einmalig auf diese Daten an (einmalig in dem Sinne, dass keine weitere Optimierung des Algorithmus durchgeführt wird) und geben Sie für jedes Bild aus, wieviele Mitosen ihre Implementierung gefunden hat und wieviele laut Dateinamen zu finden gewesen wären. Schließlich geben Sie noch die Genaugikeit ihrer Lösung für diesen Datensatz an, in dem Sie die Gesamtzahl der gefundenen Mitosen durch die Gesamtzahl der tatsächlichen Mitosen teilen. 

In [ ]:
# 1.3